In [1]:
# Autoreload packages that are modified
%load_ext autoreload
%autoreload 2

from datetime import datetime, timedelta
import glob
import os
import sys
import time

import h5py
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf
import tensorflow_probability as tfp
tfb = tfp.bijectors
tfd = tfp.distributions
tfk = tfp.math.psd_kernels

import bokeh
import bokeh.io
import bokeh.plotting
import bokeh.models
from IPython.display import display, HTML

from tqdm import tqdm
from itertools import islice

bokeh.io.output_notebook(hide_banner=True)

from data import get_data, get_prediction_dates, dt_to_float, float_to_dt

os.environ["CUDA_VISIBLE_DEVICES"]="0"

cwd = os.getcwd()
sys.path.append(f"{cwd}/forecast_rodeo")
sys.path.append(f"{cwd}/forecast_rodeo/src/experiments")
from experiments_util import get_target_date, month_day_subset
from stepwise_util import default_stepwise_candidate_predictors

# Load data and split into training set for a prediction date

In [2]:
X_original, anoms, clims, temps, dates, \
columnstr_to_index, index_to_columnstr = get_data(add_ones=False)
dates_as_float = np.array([dt_to_float(d) for d in dates]).astype(np.float64)

['forecast_rodeo/results/regression/shared/contest_tmp2m_34w/lat_lon_date_data-contest_tmp2m_34w.h5', 'forecast_rodeo/results/regression/shared/contest_tmp2m_34w/date_data-contest_tmp2m_34w.h5']
lat_oi: 42.0, lon_oi: 251.0


In [3]:
prediction_dates, prediction_dates_strs = get_prediction_dates(subsample_rate=10)

In [4]:
prediction_date = prediction_dates[0]
train_idxs = dates <= prediction_date
test_idxs = dates > prediction_date

X_original_train = X_original[train_idxs, :]
anoms_train = anoms[train_idxs]
clims_train = clims[train_idxs]
temps_train = temps[train_idxs]
dates_train = dates[train_idxs]
dates_as_float_train = dates_as_float[train_idxs]

X_original_test = X_original[test_idxs, :]
anoms_test = anoms[test_idxs]
clims_test = clims[test_idxs]
temps_test = temps[test_idxs]
dates_test = dates[test_idxs]
dates_as_float_test = dates_as_float[test_idxs]

print(f"Num in training set: {len(anoms_train)}")
print(f"Num in testing set: {len(anoms_test)}")

is_predicting_anomaly = False
if is_predicting_anomaly:
    Y_train = anoms_train.astype(np.float64)
    Y_test = anoms_test.astype(np.float64)
    print(f"Y is temperature anomaly!")
else:
    Y_train = temps_train.astype(np.float64)
    Y_test = temps_test.astype(np.float64)
    print(f"Y is temperature directly!")

Num in training set: 10693
Num in testing set: 2590
Y is temperature directly!


In [5]:
# Plot data
fig = bokeh.plotting.figure(
    width=800, height=400)
fig.xaxis.axis_label = 'Date'
fig.yaxis.axis_label = 'Temperature (C)'
fig.add_layout(bokeh.models.Title(
    text='In situ air measurements at Mauna Loa, Observatory, Hawaii',
    text_font_style="italic"), 'above')
fig.add_layout(bokeh.models.Title(
    text='Temperature (C)',
    text_font_size="14pt"), 'above')
fig.line(
    dates_as_float, temps, legend_label='All data',
    line_width=2, line_color='midnightblue')
fig.legend.location = 'top_left'
fig.toolbar.autohide = True
bokeh.plotting.show(fig)
#

# GP Model

In [6]:
# Define mean function which is the means of observations
observations_mean = tf.constant(
    [np.mean(Y_train)], dtype=tf.float64)
mean_fn = lambda _: observations_mean

# Define the kernel with trainable parameters. 
# Note we transform some of the trainable variables to ensure
#  they stay positive.

# Use float64 because this means that the kernel matrix will have 
#  less numerical issues when computing the Cholesky decomposition

# Constrain to make sure certain parameters are strictly positive
constrain_positive = tfb.Shift(np.finfo(np.float64).tiny)(tfb.Exp())

# Smooth kernel hyperparameters
smooth_amplitude = tfp.util.TransformedVariable(
    initial_value=10., bijector=constrain_positive, dtype=np.float64,
    name='smooth_amplitude')
smooth_length_scale = tfp.util.TransformedVariable(
    initial_value=10., bijector=constrain_positive, dtype=np.float64,
    name='smooth_length_scale')

# Smooth kernel
smooth_kernel = tfk.ExponentiatedQuadratic(
    amplitude=smooth_amplitude, 
    length_scale=smooth_length_scale)

# Local periodic kernel hyperparameters
periodic_amplitude = tfp.util.TransformedVariable(
    initial_value=5.0, bijector=constrain_positive, dtype=np.float64,
    name='periodic_amplitude')
periodic_length_scale = tfp.util.TransformedVariable(
    initial_value=1.0, bijector=constrain_positive, dtype=np.float64,
    name='periodic_length_scale')
periodic_period = tfp.util.TransformedVariable(
    initial_value=1.0, bijector=constrain_positive, dtype=np.float64,
    name='periodic_period')
periodic_local_length_scale = tfp.util.TransformedVariable(
    initial_value=1.0, bijector=constrain_positive, dtype=np.float64,
    name='periodic_local_length_scale')
# Local periodic kernel
local_periodic_kernel = (
    tfk.ExpSinSquared(
        amplitude=periodic_amplitude, 
        length_scale=periodic_length_scale,
        period=periodic_period) * 
    tfk.ExponentiatedQuadratic(
        length_scale=periodic_local_length_scale))

# Noise variance of observations
# Start out with a medium-to high noise
observation_noise_variance = tfp.util.TransformedVariable(
    initial_value=1, bijector=constrain_positive, dtype=np.float64,
    name='observation_noise_variance')

trainable_variables = [v.variables[0] for v in [
    smooth_amplitude,
    smooth_length_scale,
    periodic_amplitude,
    periodic_length_scale,
    periodic_period,
    periodic_local_length_scale,
    observation_noise_variance
]]

# Sum all kernels to single kernel containing all characteristics
kernel = (smooth_kernel + local_periodic_kernel)

InternalError: CUDA runtime implicit initialization on GPU:0 failed. Status: out of memory

# GP Fitting

In [ ]:
# Define mini-batch data iterator
batch_size = 256

batched_dataset = (
    tf.data.Dataset.from_tensor_slices(
        (dates_as_float_train.reshape(-1, 1), Y_train))
    .shuffle(buffer_size=len(Y_train))
    .repeat(count=None)
    .batch(batch_size)
)

@tf.function(autograph=False, experimental_compile=False)  # Use tf.function for more effecient function evaluation
def gp_loss_fn(index_points, observations):
    """Gaussian process negative-log-likelihood loss function."""
    gp = tfd.GaussianProcess(
        mean_fn=mean_fn,
        kernel=kernel,
        index_points=index_points,
        observation_noise_variance=observation_noise_variance
    )
    
    negative_log_likelihood = -gp.log_prob(observations)
    return negative_log_likelihood


In [ ]:
# Fit hyperparameters
optimizer = tf.keras.optimizers.Adam(learning_rate=0.005)

# Training loop
batch_nlls = []  # Batch NLL for plotting
full_ll = []  # Full data NLL for plotting
nb_iterations = 10001
for i, (index_points_batch, observations_batch) in tqdm(enumerate(islice(batched_dataset, nb_iterations)), file=sys.stdout):
    # Run optimization for single batch
    with tf.GradientTape() as tape:
        loss = gp_loss_fn(index_points_batch, observations_batch)
    grads = tape.gradient(loss, trainable_variables)
    optimizer.apply_gradients(zip(grads, trainable_variables))
    batch_nlls.append((i, loss.numpy()))
    # Evaluate on all observations
    if i % 100 == 0:
        # Evaluate on all observed data
        ll = gp_loss_fn(
            index_points=dates_as_float_train.reshape(-1, 1),
            observations=Y_train)
        full_ll.append((i, ll.numpy()))

In [ ]:
# Plot NLL over iterations
fig = bokeh.plotting.figure(
    width=600, height=400, 
    x_range=(0, nb_iterations),
    y_range=(400, 700)
)
fig.add_layout(bokeh.models.Title(
    text='Negative Log-Likelihood (NLL) during training', 
    text_font_size="14pt"), 'above')
fig.xaxis.axis_label = 'iteration'
fig.yaxis.axis_label = 'NLL batch data'
# First plot
fig.line(
    *zip(*batch_nlls), legend_label='Batch data',
    line_width=2, line_color='midnightblue')

# Second plot
# Setting the second y axis range name and range
fig.extra_y_ranges = {
    'fig1ax2': bokeh.models.Range1d(start=0, end=25000)}
fig.line(
    *zip(*full_ll), legend_label='All observed data',
    line_width=2, line_color='red', y_range_name='fig1ax2')
# Adding the second axis to the plot.  
fig.add_layout(bokeh.models.LinearAxis(
    y_range_name='fig1ax2', axis_label='NLL all data'), 'right')

fig.legend.location = 'top_right'
fig.toolbar.autohide = True
bokeh.plotting.show(fig)

# Analyze results

In [ ]:
# Show values of parameters found
variables = [
    smooth_amplitude,
    smooth_length_scale,
    periodic_amplitude,
    periodic_length_scale,
    periodic_period,
    periodic_local_length_scale,
    observation_noise_variance
]

data = list([(var.variables[0].name[:-2], var.numpy()) for var in variables])
df_variables = pd.DataFrame(
    data, columns=['Hyperparameters', 'Value'])
display(HTML(df_variables.to_html(
    index=False, float_format=lambda x: f'{x:.4f}')))

In [ ]:
# let's predict up to 28 days
num_forecast_days = 28
one_day = timedelta(days=1)

extra_dates = np.array([prediction_date + i * one_day for i in np.arange(1, 29)])
extra_dates_as_floats = np.array([dt_to_float(d) for d in extra_dates]).reshape(-1, 1)

prediction_dates = dates_as_float_train.reshape(-1, 1)
prediction_dates_extra = np.concatenate((prediction_dates, extra_dates_as_floats))

In [ ]:
# Posterior GP using fitted kernel and observed data
gp_posterior_predict = tfd.GaussianProcessRegressionModel(
    mean_fn=mean_fn,
    kernel=kernel,
    index_points=extra_dates_as_floats,
    observation_index_points=dates_as_float_train.reshape(-1, 1),
    observations=Y_train,
    observation_noise_variance=observation_noise_variance)

# Posterior mean and standard deviation
posterior_mean_predict = gp_posterior_predict.mean()
posterior_std_predict = gp_posterior_predict.stddev()

In [ ]:
# Calculate the skill!
anoms_hat = posterior_mean_predict.numpy()[14:] - clims_test[14:num_forecast_days]
anoms_gt = anoms_test[14:num_forecast_days]

def calculate_skill(a_hat, a):
    return a_hat.dot(a) / (np.linalg.norm(a_hat) * np.linalg.norm(a))

print(calculate_skill(anoms_hat.squeeze(), anoms_gt.squeeze()))


In [ ]:
# Plot posterior predictions

# Get posterior predictions
μ = posterior_mean_predict.numpy()
σ = posterior_std_predict.numpy()

# Plot
fig = bokeh.plotting.figure(
    width=800, height=400, 
    x_range = (2010, 2012),
    y_range = (0, 40))
fig.xaxis.axis_label = 'Date'
fig.yaxis.axis_label = 'Temperature (C)'
fig.add_layout(bokeh.models.Title(
    text='Posterior predictions conditioned on observations before prediction date.',
    text_font_style="italic"), 'above')
fig.add_layout(bokeh.models.Title(
    text='Temperature Forecast', 
    text_font_size="14pt"), 'above')
fig.circle(
    dates_as_float, temps, legend_label='True data',
    size=2, line_color='midnightblue')
fig.line(
    extra_dates_as_floats.squeeze(), μ, legend_label='μ (predictions)',
    line_width=2, line_color='firebrick')

# Prediction interval
band_x = np.append(
    extra_dates_as_floats.squeeze(), extra_dates_as_floats.squeeze()[::-1])
band_y = np.append(
    (μ + 2*σ), (μ - 2*σ)[::-1])
fig.patch(
    band_x, band_y, color='firebrick', alpha=0.4, 
    line_color='firebrick', legend_label='2σ')

# vertical line showing predicition location
fig.line(
    [dt_to_float(prediction_date) for _ in range(100)], np.arange(0, 100, 1), legend_label='μ (predictions)',
    line_width=2, line_dash="dashed", line_color='darkorange')

fig.legend.location = 'top_left'
fig.toolbar.autohide = True
bokeh.plotting.show(fig)
#


## GP Decomposition

In [ ]:
# Posterior GP using fitted kernel and observed data
smooth_kernel_gp_posterior_predict = tfd.GaussianProcessRegressionModel(
    mean_fn=mean_fn,
    kernel=smooth_kernel,
    index_points=prediction_dates_extra,
    observation_index_points=dates_as_float_train.reshape(-1, 1),
    observations=Y_train,
    observation_noise_variance=observation_noise_variance)
local_periodic_kernel_gp_posterior_predict = tfd.GaussianProcessRegressionModel(
    mean_fn=mean_fn,
    kernel=local_periodic_kernel,
    index_points=prediction_dates_extra,
    observation_index_points=dates_as_float_train.reshape(-1, 1),
    observations=Y_train,
    observation_noise_variance=observation_noise_variance)

# Posterior mean and standard deviation
smooth_kernel_posterior_mean_predict = smooth_kernel_gp_posterior_predict.mean()
smooth_kernel_posterior_std_predict = smooth_kernel_gp_posterior_predict.stddev()

local_periodic_kernel_posterior_mean_predict = local_periodic_kernel_gp_posterior_predict.mean()
local_periodic_kernel_posterior_std_predict = local_periodic_kernel_gp_posterior_predict.stddev()


# Plot posterior predictions

# Get posterior predictions
smooth_kernel_μ = smooth_kernel_posterior_mean_predict.numpy()
smooth_kernel_σ = smooth_kernel_posterior_std_predict.numpy()

local_periodic_kernel_μ = local_periodic_kernel_posterior_mean_predict.numpy()
local_periodic_kernel_σ = local_periodic_kernel_posterior_std_predict.numpy()

# Plot
fig = bokeh.plotting.figure(
    width=800, height=400)
#     x_range = (2010, 2012),
#     y_range = (0, 40))
fig.xaxis.axis_label = 'Date'
fig.yaxis.axis_label = 'Temperature (C)'
fig.add_layout(bokeh.models.Title(
    text='Posterior predictions conditioned on observations before prediction date.',
    text_font_style="italic"), 'above')
fig.add_layout(bokeh.models.Title(
    text='Temperature Forecast', 
    text_font_size="14pt"), 'above')
fig.circle(
    dates_as_float, temps, legend_label='True data',
    size=2, line_color='midnightblue')

# smooth kernel
fig.line(
    prediction_dates_extra.squeeze(), smooth_kernel_μ, legend_label='smooth kernel μ (predictions)',
    line_width=2, line_color='firebrick')
# Prediction interval
band_x = np.append(
    prediction_dates_extra.squeeze(), prediction_dates_extra.squeeze()[::-1])
band_y = np.append(
    (smooth_kernel_μ + 2*smooth_kernel_σ), (smooth_kernel_μ - 2*smooth_kernel_σ)[::-1])
fig.patch(
    band_x, band_y, color='firebrick', alpha=0.4, 
    line_color='firebrick', legend_label='smooth kernel 2σ')


# periodic kernel
fig.line(
    prediction_dates_extra.squeeze(), local_periodic_kernel_μ, legend_label='periodic kernel μ (predictions)',
    line_width=2, line_color='forestgreen')
# Prediction interval
band_x = np.append(
    prediction_dates_extra.squeeze(), prediction_dates_extra.squeeze()[::-1])
band_y = np.append(
    (local_periodic_kernel_μ + 2*local_periodic_kernel_σ), (local_periodic_kernel_μ - 2*local_periodic_kernel_σ)[::-1])
fig.patch(
    band_x, band_y, color='forestgreen', alpha=0.4, 
    line_color='forestgreen', legend_label='periodic kernel 2σ')


fig.legend.location = 'top_left'
fig.toolbar.autohide = True
bokeh.plotting.show(fig)
#
